In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

*Get the raw html text from the designated wikipedia page*

In [2]:
raw_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

*Use beautifulSoup to format and read the text of the html and segment it to show only the table area and subsequently cycle through the table data (td)*

In [3]:
soup_data = BeautifulSoup(raw_data, 'html5lib')

In [4]:
soup_table = soup_data.find('table')

In [5]:
soup_table.findAll('td')[1]

<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>

*As per guidance in the summary of the submission add the details from the table to a pandas dataframe and then format some of the borough location names so that they read easier*

In [6]:
table_contents = []

for row in soup_table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


In [7]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

*Look at data to see that it has worked correctly*

In [8]:
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


*Final cell must display the shape of the data as per instructions*

In [9]:
df.shape

(103, 3)

In [10]:
#Geocoder not working
#use csv

lat_lng = pd.read_csv('Downloads\Geospatial_Coordinates.csv')

lat_lng.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

lat_lng.head(5)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


*Join both dataframes together*

In [11]:
df_full = df.join(lat_lng.set_index('PostalCode'), on='PostalCode')
df_full.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [12]:
CLIENT_ID = 'OASH20JYAO5LPBW3LHXBG41PUMLO2B5VPRUDKG4JQAVSZRUH' # your Foursquare ID
CLIENT_SECRET = 'CSG0HIV24SLZ1J2FX5MLJLPZVKQWB3GW0YUHICXJJO0O1EYH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 5 

*After entering in the required details for the 4square API and setting the limit on results received due to internet and call limitations, the radius is set to 500 and the venue details are pulled for each postal code*

In [13]:
radius = 500 # defined radius

df_venues = []

for name, lat, lng in zip(df_full['PostalCode'], df_full['Latitude'], df_full['Longitude']):

    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)


    sq_results = requests.get(url).json()["response"]['groups'][0]['items']
    
    df_venues.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in sq_results])

    df_full_venues = pd.DataFrame([item for venue_list in df_venues for item in venue_list])
    df_full_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
print(df_full_venues.shape)
df_full_venues.head(5)

(434, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
1,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [14]:
df_full_venues.groupby('PostalCode').count()

,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,1,1,1,1,1,1
M1C,2,2,2,2,2,2
M1E,5,5,5,5,5,5
M1G,4,4,4,4,4,4
M1H,5,5,5,5,5,5
...,...,...,...,...,...,...
M9N,2,2,2,2,2,2
M9P,5,5,5,5,5,5
M9R,3,3,3,3,3,3


*In a similar manner to the analysis in the lab, one hot encoding is used to populate a column per every type of venue category so that a groupby can produce a probability value for each venue category per Postal Code*

In [15]:
# one hot encoding
df_onehot = pd.get_dummies(df_full_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postal Code column back to dataframe
df_onehot['PostalCode'] = df_full_venues['PostalCode'] 

# move Postal Code column to the first column
cols = list(df_onehot.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('PostalCode')) #Remove postalcode from list
df_onehot = df_onehot[['PostalCode'] + cols]

df_onehot.head()

,PostalCode,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


*get the mean result per venue category for each postal code*

In [16]:
df_grouped = df_onehot.groupby('PostalCode').mean().reset_index()
df_grouped.head(5)

,PostalCode,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


*Run KMeans clustering with the number of clusters set to 5*

In [17]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5
df_grouped_clustering = df_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 0, 1, 3, 1, 1, 1])

*Attach the cluster labels to the combined dataframe so that we can can populate the folium map with all necessary details*

In [18]:
# add clustering labels
df_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df_full_venues

# merge df_grouped with df_merged to add latitude/longitude for each code
df_merged = df_merged.join(df_grouped.set_index('PostalCode'), on='PostalCode')

df_merged.head()


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,Airport,Airport Food Court,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop,4,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
!conda install -c conda-forge folium=0.5.0 --yes
#Installing the necessary folium package


Solving environment: ...working... done

# All requested packages already installed.



*Import necessary packages and assign colours to clusters before plotting on a map to visualise the relationships for the clusters*

In [45]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

lat_start = 43.651070 
lon_start = -79.347015

# create map
map_clusters = folium.Map(location=[lat_start, lon_start], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map with additional callouts
markers_colors = []
for lat, lon, poi, cluster, ven in zip(df_merged['Venue Latitude'], df_merged['Venue Longitude'], df_merged['PostalCode'], df_merged['Cluster Labels'], df_merged['Venue Category']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + ' ' + str(ven), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

*The below cell was used to investigate each cluster in coordination with the map above to try an identify the defining characteristic for the cluster*

In [41]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,PostalCode Latitude,Venue Longitude,Venue Category,Cluster Labels,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,...,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
7,43.654260,-79.361809,Coffee Shop,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,43.654260,-79.362017,Bakery,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,43.654260,-79.358008,Distribution Center,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,43.654260,-79.359874,Spa,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,43.654260,-79.356980,Restaurant,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,43.628841,-79.518408,Burger Joint,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
430,43.628841,-79.518627,Discount Store,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
431,43.628841,-79.518495,Supplement Shop,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0
432,43.628841,-79.518169,Wings Joint,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


# Findings

## Cluster 1

*Cluster 1 appears to dominate Toronto Central being comprised mainly of eateries such as restaurants and cafes*

## Cluster 2

*Cluster 2 has no presence in central Toronto and is only found on the outskirts in the North East and West and seem to be typically characterised by small outlets containing walk in services such as a bank*

## Cluster 3

*Cluster 3 is located throughout and appears to capture metropolitan services such as parks and Coffee Shops*

## Cluster 4

*Cluster 4 seems to capture similar public utlities as cluster 3 but it less dense areas, typically capturing less than 5 local venues in a cluster*

## Cluster 5

*Cluster 5 is the smallest of all the clusters and is only found at relatively remote locations (no other venues), they're baseball stadiums which makes sense given the area and parking required for a baseball stadium which would preclude any densely populated business areas*